This notebook is a copy for testing further speedups of the simulation.

In [3]:
from tqdm.notebook import trange
import pathlib
import numpy as np
from scipy.ndimage import convolve

In [4]:
#Random number generator that outputs uniformly distributed floats in the range [0,1)
rng = np.random.default_rng()

<h1>Assignment 2:</h1>

<h2>Simulation:</h2>

<h3>Calculating the parameters:</h3>

The Hamiltonian is given by $$H = -J \sum_{\left\langle ij \right\rangle}{s_i s_j}$$ where $s_i \in \{+1,-1\}$.

We need to find a suitable value of $J$ such that the critical temperature $T_c$ as given by the mean field theory approximation is $2$. The relation between $J$ and $T_c$ in the mean field theory approximation is given as $JC\beta_c = \frac{JC}{kT_c}=1$ where $C=4$ for the 2D Ising model. We will define our units such that $k=2$. This conveniently gives us $J=1$.

We will consider periodic boundary conditions.

<h3>Monte-Carlo step:</h3>

The spin-up state is taken as $+1$ and the spin-down state is taken as $-1$.

In one Monte-Carlo step, we flip the state of one spin and calculate the resulting energy difference $\Delta E = E_f - E_i$. When a spin is flipped, only $4$ terms are affected in the Hamiltonian, and they are the interactions between the nearest neighbour spins of the flipped spin and the flipped spin itself. So, instead of recalculating the entire Hamiltonian every time, we can simply calculate the energy difference from these $4$ terms.

Let us say spin $s_{(i,j)}$ was flipped (which is the spin at row $i$ and column $j$). The energy difference is then $$\Delta E = 2Js_{(i,j)} \cdot \left( s_{(i+1,j)} + s_{(i,j+1)} + s_{(i-1,j)} + s_{(i,j-1)} \right)$$ since we just flipped the sign of these $4$ terms. Note that the indices are considered modulo $N$ because of periodic boundary conditions.

Then we can test if the new state satisfies our conditions and if it does, we'll flip the spin.

<h3>Speeding up the simulation:</h3>

If we consider the lattices for fixed $N$, in each timestep we are performing basically the same operation on all of them, and each step follows the same sequence of spins. So instead of evolving them one by one, we can process all of them at the same time. NumPy does this effectively through "vectorization". In the following code, I have used this idea. Instead of passing one lattice-temperature pair, I have passed lists of lattice-temperature pairs. Now, in each step, I can calculate $\Delta E$, whether the state was accepted, and flipping the spin if the state was accepted, for all of the lattices in one go. The resulting speedup is huge, I estimate about $10$ times faster.

We have to keep $N$ fixed, otherwise the array indexing would get messed up when calculating $\Delta E$.

We can make further optimisations. There is a $2$ in the energy difference formula and we also defined $k=2$. These can be cancelled to remove two multiplications. Also, division is much slower than multiplication, so we can calculate $\frac{1}{T}$ beforehand and multiply by it instead of dividing in every step.

In [3]:
def mc_step(lattices,N,rands,stepper,i,j):
    #Calculate del_E
    del_Es = lattices[:,i,j]*(lattices[:,i-1,j]+lattices[:,i+1-N,j]+lattices[:,i,j-1]+lattices[:,i,j+1-N])
    #Check if state is accepted
    rejects = del_Es - rands[stepper,:]
    #Flip if accepted
    #Reject value is + or -, so map +ve -> 1 (no flip) and -ve -> -1 (flip)
    lattices[:,i,j] *= np.sign(rejects).astype(np.int8)

In [4]:
def mc_step_special(lattices,down_rights,N,rands,stepper,i,j):
    #Calculate del_E
    del_Es = lattices[:,i,j]*(lattices[:,i-1,j]+lattices[:,i+1-N,j])+down_rights[:,i,j]
    #Check if state is accepted
    rejects = del_Es - rands[stepper,:]
    #Flip if accepted
    #Reject value is + or -, so map +ve -> 1 (no flip) and -ve -> -1 (flip)
    lattices[:,i,j] *= np.sign(rejects).astype(np.int8)

In [5]:
def mc_timestep(lattices,N,rands,stepper,samples):
    mask = np.array([[0,1,0],
                     [1,0,1],
                     [0,1,0]])
    down_rights = lattices*np.array([convolve2d(lattice,mask,mode='same',boundary='wrap') for lattice in lattices])
    #Run one step for each spin in the lattice
    for i in range(0,N-1,2):
        for j in range(0,N-1,2):
            mc_step_special(lattices,down_rights,N,rands,stepper,i,j)
            stepper += 1
            if stepper >= samples:
                stepper = 0
                rands = np.transpose(rng.exponential(scale=Ts_scale,size=(Ts.shape[0],samples)))
    for i in range(1,N-1,2):
        for j in range(1,N-1,2):
            mc_step_special(lattices,down_rights,N,rands,stepper,i,j)
            stepper += 1
            if stepper >= samples:
                stepper = 0
                rands = np.transpose(rng.exponential(scale=Ts_scale,size=(Ts.shape[0],samples)))
    for i in range(N-1):
        mc_step(lattices,N,rands,stepper,i,N-1)
        stepper += 1
        if stepper >= samples:
            stepper = 0
            rands = np.transpose(rng.exponential(scale=Ts_scale,size=(Ts.shape[0],samples)))
    for j in range(N-1):
        mc_step(lattices,N,rands,stepper,N-1,j)
        stepper += 1
        if stepper >= samples:
            stepper = 0
            rands = np.transpose(rng.exponential(scale=Ts_scale,size=(Ts.shape[0],samples)))
    mc_step(lattices,N,rands,stepper,N-1,N-1)
    stepper += 1
    if stepper >= samples:
        stepper = 0
        rands = np.transpose(rng.exponential(scale=Ts_scale,size=(Ts.shape[0],samples)))
    return stepper

In [5]:
def mc_timestep_even(lattices,Ts_scale,nn_mask,black_mask,white_mask):
    rand = rng.exponential(scale=Ts_scale,size=lattices.shape)
    rand_black = rand*black_mask
    rand_white = rand*white_mask
    #Black
    del_Es = lattices*np.array([convolve(lattice,nn_mask,mode='wrap') for lattice in lattices])
    del_Es_black = del_Es*black_mask
    rejects_black = del_Es_black - rand_black + white_mask
    lattices *= np.sign(rejects_black).astype(np.int8)
    #White
    del_Es = lattices*np.array([convolve(lattice,nn_mask,mode='wrap') for lattice in lattices])
    del_Es_white = del_Es*white_mask
    rejects_white = del_Es_white - rand_white + black_mask
    lattices *= np.sign(rejects_white).astype(np.int8)

In [6]:
def mc_timestep_odd(lattices,Ts_scale,nn_mask,black_mask,white_mask):
    rand = rng.exponential(scale=Ts_scale,size=lattices.shape)
    rand_black = rand[:,:-1,:-1]*black_mask
    rand_white = rand[:,:-1,:-1]*white_mask
    #Black
    del_Es = lattices[:,:-1,:-1]*np.array([convolve(lattice,nn_mask,mode='wrap') for lattice in lattices])[:,:-1,:-1]
    del_Es_black = del_Es*black_mask
    rejects_black = del_Es_black - rand_black + white_mask
    lattices[:,:-1,:-1] *= np.sign(rejects_black).astype(np.int8)
    #White
    del_Es = lattices[:,:-1,:-1]*np.array([convolve(lattice,nn_mask,mode='wrap') for lattice in lattices])[:,:-1,:-1]
    del_Es_white = del_Es*white_mask
    rejects_white = del_Es_white - rand_white + black_mask
    lattices[:,:-1,:-1] *= np.sign(rejects_white).astype(np.int8)
    #Process last row and column normally
    for i in range(N-1):
        del_E = lattices[:,i,N-1]*(lattices[:,i-1,N-1]+lattices[:,i+1-N,N-1]+lattices[:,i,N-2]+lattices[:,i,0])
        reject = del_E - rand[:,i,N-1]
        lattices[:,i,N-1] *= np.sign(reject).astype(np.int8)
    for j in range(N):
        del_E = lattices[:,N-1,j]*(lattices[:,N-1,j-1]+lattices[:,N-1,j+1-N]+lattices[:,N-2,j]+lattices[:,0,j])
        reject = del_E - rand[:,N-1,j]
        lattices[:,i,N-1] *= np.sign(reject).astype(np.int8)

In [77]:
N = 40
Ts = np.linspace(0.1,4,40) #List of temperatures to compute for
#Create lattices
lattices = np.ones(shape=(Ts.shape[0],N,N),dtype=np.int8)
Ts_scale = np.array([T*np.ones(shape=(N,N)) for T in Ts])
nn_mask = np.array([[0,1,0],
                    [1,0,1],
                    [0,1,0]]).astype(np.int8)
black_mask = np.tile([[1,0],[0,1]],(lattices.shape[0],N//2,N//2)).astype(np.int8)
white_mask = np.tile([[0,1],[1,0]],(lattices.shape[0],N//2,N//2)).astype(np.int8)
rand_black_mask = np.broadcast_to(np.tile([[1,0],[0,1]],(N//2,N//2)),shape = (Ts.shape[0],N,N))
rand_white_mask = np.broadcast_to(np.tile([[0,1],[1,0]],(N//2,N//2)),shape = (Ts.shape[0],N,N))
%timeit mc_timestep_even(lattices,Ts_scale,nn_mask,black_mask,white_mask,rand_black_mask,rand_white_mask)

# xyxy
# yxyx
# xyxy
# yxyx
# if n is even:
# colour as a checkerboard,then states can be evolved independently
# then recompute and compute the other states

# xyxyx
# yxyxy
# xyxyx
# yxyxy
# xyxyx
# if n is odd:
# what to do...
# use n=even algorithm on n-1 x n-1 and then compute the last row and column normally
# maybe...


5.83 ms ± 61.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


<h3>Simulating and gathering snapshots:</h3>

The following cells define the required elements of the simulation and then we run it. We take an $N \times N$ sized lattice at temperature $T$. For the initial state, we'll consider all spins to point up. First it is allowed to equilibriate for ```t_equilibrium``` timesteps, then ```snapshot_count``` snapshots are gathered every ```snapshot_interval``` timesteps.

Then the snapshots are stored in CSV format for further analysis. This way, we have to run the simulation just once, and later we can use the stored data anytime.

In [20]:
def simulate(N,Ts,equilibriation_time,autocorrelation_time,snapshot_count):
    #Create lattices
    lattices = np.ones(shape=(Ts.shape[0],N,N),dtype=np.int8)
    Ts_scale = np.array([T*np.ones(shape=(N,N)) for T in Ts])
    nn_mask = np.array([[0,1,0],
                        [1,0,1],
                        [0,1,0]]).astype(np.int8)
    black_mask = np.tile([[1,0],[0,1]],(Ts.shape[0],N//2,N//2)).astype(np.int8)
    white_mask = np.tile([[0,1],[1,0]],(Ts.shape[0],N//2,N//2)).astype(np.int8)
    timestep_method = None
    if N%2 == 0:
      timestep_method = mc_timestep_even
    else:
      timestep_method = mc_timestep_odd

    #Allow lattices to equilibriate
    for t in trange(equilibriation_time,desc='Equilibriation'):
        timestep_method(lattices,Ts_scale,nn_mask,black_mask,white_mask)

    #Collect snapshots
    snapshots = np.empty((snapshot_count,*lattices.shape),dtype=np.int8)
    for i in trange(snapshot_count,desc='Collecting snapshots'):
        for t in range(autocorrelation_time):
            timestep_method(lattices,Ts_scale,nn_mask,black_mask,white_mask)
        snapshots[i] = lattices

    temp_ordered_snaps = np.moveaxis(snapshots,1,0)
    np.savez_compressed(f'data_{N}.npz',*temp_ordered_snaps)

In [8]:
#Set the parameters here
t_equilibrium = 10000 #Equilibriation time
snapshot_interval = 100 #Autocorrelation time
snapshot_count = 10000 #Number of snapshots
N = 40 #Lattice size
T_list = np.linspace(0.1,4,40) #List of temperatures to compute for

In [ ]:
#Run this cell to generate the data
print(f'Processing N = {N}...')
simulate(N,T_list,t_equilibrium,snapshot_interval,snapshot_count)

<hr>

In [35]:
%timeit simulate(16,np.array([1]),1,0,0)

Equilibriation:   0%|          | 0/1 [00:00<?, ?it/s]

Equilibriation:   0%|          | 0/1 [00:00<?, ?it/s]

Equilibriation:   0%|          | 0/1 [00:00<?, ?it/s]

Equilibriation:   0%|          | 0/1 [00:00<?, ?it/s]

Equilibriation:   0%|          | 0/1 [00:00<?, ?it/s]

...

Equilibriation:   0%|          | 0/1 [00:00<?, ?it/s]

Equilibriation:   0%|          | 0/1 [00:00<?, ?it/s]

Equilibriation:   0%|          | 0/1 [00:00<?, ?it/s]

Equilibriation:   0%|          | 0/1 [00:00<?, ?it/s]

45.8 ms ± 7.34 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
